# **Files/Models to be located for this code**

1.   Machine Learning model file named as model.h5
2.   shape_predictor_68_face_landmarks.dat mode file to detect whole face
3.   **Provind input image in filter_image function the 1st contact function**

# **Packages to import**
* dlib, cv2, numpy, os, pandas, keras( from tensorflow)**

















In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
from tensorflow import keras
import pandas as pd
import os
import numpy as np
import cv2
import dlib

from google.colab.patches import cv2_imshow

**Initialising model using model.h5 file**

In [4]:
model_path="/content/drive/MyDrive/14th Red Eye Detection and Removal-20230808T161401Z-001/14th Red Eye Detection and Removal/model.h5"
model = keras.models.load_model(model_path)

**Standard size of cropped eye not whole image on which model trained is 32 X 32 pixels**

In [5]:
dim=(32,32)

**Function eye_finder() to detect eyes and and crop that area of image**

In [6]:
def eye_finder(img):
  return_eyes=[]
  eyes_dimension=[]
  eyes_shape=[]
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor('/content/drive/MyDrive/14th Red Eye Detection and Removal-20230808T161401Z-001/14th Red Eye Detection and Removal/shape_predictor_68_face_landmarks.dat')
  if img.shape[0]>600 and img.shape[1]>600:
    img=cv2.resize(img,None,fx=0.40,fy=0.40)
  real_img=img.copy()

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  rects = detector(gray, 1)
  shape_list=[]
  def shape_to_np(shape, dtype="int"):
      coords = np.zeros((68, 2), dtype=dtype)
      for i in range(0, 68):
          coords[i] = (shape.part(i).x, shape.part(i).y)
      return coords
  for (i, rect) in enumerate(rects):
      shape = predictor(gray, rect)
      shape = shape_to_np(shape)
      shape_list.append(shape)

  for shape in shape_list:
    ## Getting eyes points from predictor shape by model _______________________X_________________________________________________________
    ## left = [36, 37, 38, 39, 40, 41]
    ## right = [42, 43, 44, 45, 46, 47]

    left_eye=[]
    right_eye=[]
    left_eye=shape[36:42]
    right_eye=shape[42:48]

    ## Eye coordinate calculation in image  _________________________________________________X_____________________________________________
    left_x=1000
    left_y=1000
    left_w=-1
    left_h=-1
    right_x=1000
    right_y=1000
    right_w=-1
    right_h=-1

    for i in range(6):
      left_x=min(left_x,left_eye[i][0])
      left_y=min(left_y,left_eye[i][1])
      left_w=max(left_w,left_eye[i][0])
      left_h=max(left_h,left_eye[i][1])

      right_x=min(right_x,right_eye[i][0])
      right_y=min(right_y,right_eye[i][1])
      right_w=max(right_w,right_eye[i][0])
      right_h=max(right_h,right_eye[i][1])

    ## Eyes cropping from Real image  _________________________________________________X_____________________________________________
    extra=3

    left_start_point=(left_x-extra, left_y-extra)
    left_end_point=(left_w+extra, left_h+extra)

    right_start_point=(right_x-extra, right_y-extra)
    right_end_point=(right_w+extra, right_h+extra)


    left_eye_box = real_img[left_y-extra:left_h+extra,left_x-extra:left_w+extra]
    right_eye_box = real_img[right_y-extra:right_h+extra,right_x-extra:right_w+extra]

    eyes_dimension.append([left_y-extra,left_h+extra,left_x-extra,left_w+extra])
    eyes_dimension.append([right_y-extra,right_h+extra,right_x-extra,right_w+extra])
    eyes_shape.append(left_eye_box.shape)
    eyes_shape.append(right_eye_box.shape)

    ## resizing the eyes image  to 32 X 32
    dim=(32,32)
    left_inlarge_img=cv2.resize(left_eye_box,dim,interpolation=cv2.INTER_AREA)
    right_inlarge_img=cv2.resize(right_eye_box,dim,interpolation=cv2.INTER_AREA)

    return_eyes.append(left_inlarge_img)
    return_eyes.append(right_inlarge_img)

  return (return_eyes,eyes_dimension,eyes_shape)

**Prediction function using our model.h5 file**

In [1]:
def prediction_for_red_eye(img):
  resized_image = img.reshape(1,img.shape[0],img.shape[1],img.shape[2])
  pred= model.predict([resized_image])
  pred_output = False
  if pred>0.5:
    # print(1,"red-eye-detected")
    pred_output=True
  else:
    pred_output=True
    # print(0,"no red-eye")
  return pred_output

**Function to remove red colour from eyes**

In [2]:
def remove_red_eye(eyeImage):
  b, g ,r = cv2.split(eyeImage)
  bg = cv2.add(b,g)
  mask  = ( (r>(bg-20)) & (r>80) ).astype(np.uint8)*255
  contours, _ = cv2. findContours(mask.copy() ,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE )  # It return contours and Hierarchy
  maxArea = 0
  maxCont = None
  for cont in contours:
      area = cv2.contourArea(cont)
      if area > maxArea:
          maxArea = area
          maxCont = cont
  mask = mask * 0
  cv2.drawContours(mask , [maxCont],0 ,(255),-1 )
  mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_DILATE,(5,5)) )
  mask = cv2.dilate(mask , (3,3) ,iterations=3)
  mean  = bg /2
  int_mean=mean.astype(np.uint8)
  int_mask=mask.astype(np.uint8)
  mean = cv2.bitwise_and(int_mean,int_mask)
  mean  = cv2.cvtColor(mean ,cv2.COLOR_GRAY2BGR )
  mask = cv2.cvtColor(int_mask ,cv2.COLOR_GRAY2BGR )
  eye = cv2.bitwise_and(~mask,eyeImage)+mean
  return eye

**Function to call all functions to perform all task in order and merge the final improved eye cropped image to original image**

In [3]:
def filter_image(img):
  org_img=img.copy()

  ## Finding all the eyes in the image with there dimension/shape and location in the image ________________X_________________________________________________________________________
  eyes_list,eyes_dimension,eyes_shape = eye_finder(img)

  red_eyes=[]
  eye_count=0
  new_eye_dimension=[]
  new_eye_shape=[]

  ## Predicting red eye for eac eye in the image and making an list to process red eyes further ______________X________________________________________________________________________
  for eyes in eyes_list:
    if prediction_for_red_eye(eyes):
      red_eyes.append(eyes)
      new_eye_dimension.append(eyes_dimension[eye_count])
      new_eye_shape.append(eyes_shape[eye_count])
    eye_count=eye_count+1

  ## Red eye detected list will be processed below to remove red color _______________________________________X________________________________________________________________________
  improve_eye_list=[]
  for eye in red_eyes:
    improve_eye=remove_red_eye(eye)
    improve_eye_list.append(improve_eye)

  ## Improved Red eye is merged again on the original image _________________________________________________X________________________________________________________________________
  new_eye_count=0
  for improve_eye in improve_eye_list:
    org_shape=(new_eye_shape[new_eye_count][1],new_eye_shape[new_eye_count][0])
    improve_eye = cv2.resize(improve_eye,org_shape,interpolation=cv2.INTER_AREA)
    org_img[new_eye_dimension[new_eye_count][0]:new_eye_dimension[new_eye_count][1],new_eye_dimension[new_eye_count][2]:new_eye_dimension[new_eye_count][3]] = improve_eye
    new_eye_count=new_eye_count+1

  return org_img

# **filter_image the function that is to hit to start program**

*   INPUT - Type(img buffer), Name [ **named in code as img** ]
*   OUTPUT - Type(img buffer), Name [ **named in code as new_img** ]
*   INTERNAL FUNCTION CALLS : eye_finder(img) **=>** prediction_for_red_eye(eyes) **=>** remove_red_eye(eye)




In [4]:
# 1st contact function

path="/content/drive/MyDrive/14th Red Eye Detection and Removal-20230808T161401Z-001/14th Red Eye Detection and Removal/data/1/1.jfif"
img = cv2.imread(path)

NameError: ignored

In [ ]:
cv2_imshow(img)

In [ ]:
# main function to use for api
new_img = filter_image(img)

In [ ]:
cv2_imshow(new_img)